## ANOVA 사후검증

### Scheffé Method


In [ ]:
# 데이터 생성: 세 개의 그룹(A, B, C)에서 샘플을 생성합니다.
# ANOVA 수행: statsmodels를 사용하여 ANOVA를 수행합니다.
# Scheffe 사후검증 함수 정의: Scheffe 검증을 수행하는 함수를 정의합니다.
# Scheffe 사후검증 수행: 정의한 함수를 사용하여 사후검증을 수행하고 결과를 출력합니다.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import itertools
import scipy.stats as stats

# 데이터 생성
np.random.seed(0)
data = {
    'group': np.repeat(['A', 'B', 'C'], 20),
    'value': np.concatenate([np.random.normal(loc, 1, 20) for loc in [1, 2, 3]])
}

df = pd.DataFrame(data)

# ANOVA 수행
model = ols('value ~ group', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# Scheffe 사후검증 함수 정의
def scheffe_test(data, group_col, value_col, alpha=0.05):
    # 그룹별 평균 계산
    group_means = data.groupby(group_col)[value_col].mean()
    grand_mean = data[value_col].mean()
    
    # ANOVA의 Mean Squared Error 계산
    mse = anova_table['sum_sq']['Residual'] / anova_table['df']['Residual']
    
    # 그룹 수 및 총 샘플 수 계산
    n_groups = data[group_col].nunique()
    n_total = data.shape[0]
    
    # Scheffe 임계값 계산
    scheffe_critical_value = stats.f.ppf(1 - alpha, n_groups - 1, n_total - n_groups) * (n_groups - 1)
    
    # 쌍별 비교
    comparisons = list(itertools.combinations(group_means.index, 2))
    results = []
    
    for (i, j) in comparisons:
        mean_diff = abs(group_means[i] - group_means[j])
        se_diff = np.sqrt(mse * (1/data[data[group_col] == i].shape[0] + 1/data[data[group_col] == j].shape[0]))
        f_value = (mean_diff**2) / (se_diff**2)
        
        result = {
            'Comparison': f'{i} vs {j}',
            'Mean Difference': mean_diff,
            'F Value': f_value,
            'Significant': f_value > scheffe_critical_value
        }
        results.append(result)
    
    return pd.DataFrame(results)

# Scheffe 사후검증 수행
scheffe_results = scheffe_test(df, 'group', 'value')
print(scheffe_results)


             sum_sq    df         F    PR(>F)
group     10.737434   2.0  5.268739  0.007951
Residual  58.081616  57.0       NaN       NaN
  Comparison  Mean Difference    F Value  Significant
0     A vs B         0.486416   2.321942        False
1     A vs C         1.035582  10.524596         True
2     B vs C         0.549167   2.959677        False
